In [1]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import heapq as hq

import numpy as np


In [2]:
PROBLEM_SIZE = 50 #5,10,50,100,500,1000
NUM_SETS = 100 #10,20,100,200,1000,2000
P_TILE = 0.05 #0.3,0.2,0.05,0.03,0.05,0.03
SETS = tuple(np.array([random() < P_TILE for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)) 
State = namedtuple('State', ['taken', 'not_taken'])

def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [11]:

frontier = LifoQueue() #DFS
frontier.put(State(set(), set(range(NUM_SETS))))

counter = 0
current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put(new_state)
    current_state = frontier.get()

print(current_state)
print(f"Solved in {counter:,} steps")

#ex 153, 153, 0.8s


State(taken={10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9})
Solved in 90 steps


In [3]:
#greedy version
def get_priority(state):
    set_covered=reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))
    return PROBLEM_SIZE-np.count_nonzero(set_covered)

frontier = PriorityQueue()
initial_state = State(set(), set(range(NUM_SETS)))
frontier.put((0,initial_state))

counter=0
current_state = frontier.get()[1]
priority = PROBLEM_SIZE

while priority > 0:
    counter += 1
    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put((get_priority(new_state), new_state))
    priority_and_state = frontier.get()
    current_state=priority_and_state[1]
    priority=priority_and_state[0]

print(current_state)
print(len(current_state.taken))
print(f"Solved in {counter:,} steps")
print(goal_check(current_state))

greedy_best = len(current_state.taken)
#ex 31, 31, 0.5s

#2m 1s

#len=23 we want an even less expensive (long) solution -> 

State(taken={96, 1, 2, 32, 3, 9, 10, 41, 12, 14, 47, 80, 87, 55, 24, 29, 31}, not_taken={0, 4, 5, 6, 7, 8, 11, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99})
17
Solved in 17 steps
True


In [9]:
#greedy cost of action (cost of walking through an arch)
#fail
def get_priority(action):
    return -np.count_nonzero(SETS[action])

frontier = PriorityQueue()
initial_state = State(set(), set(range(NUM_SETS)))
frontier.put((0,initial_state))

counter=0
current_state = frontier.get()[1]

while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put((get_priority(action), new_state))
    priority_and_state = frontier.get()
    current_state=priority_and_state[1]

print(current_state)
print(f"Solved in {counter:,} steps")

KeyboardInterrupt: 

In [ ]:

frontier = SimpleQueue() #BFS (guarantees optimal solution, slowest)
initial_state = State(set(), set(range(NUM_SETS)))
frontier.put(initial_state)

counter = 0
current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put(new_state)
    current_state = frontier.get()


print(current_state.taken)
print(f"len={len(current_state.taken)}")
print(f"Solved in {counter:,} steps")


#ex 1845, 108, 11.5s



In [5]:

#can introduce an exponential in cost that grows really faster aftwer the len of solution of greedy algorithm

def g(state):
    length = len(state.taken)
    return length

def h(state):
    set_covered=reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))
    return PROBLEM_SIZE-np.count_nonzero(set_covered)

frontier = PriorityQueue()
initial_state = State(set(), set(range(NUM_SETS)))
frontier.put((0,(initial_state,PROBLEM_SIZE)))

counter=0
current_state = frontier.get()[1][0]
exp_cost_h = PROBLEM_SIZE

while exp_cost_h > 0:
    counter += 1
    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})

        act_cost_g = g(current_state)
        exp_cost_h = h(current_state)
        priority = act_cost_g+exp_cost_h
        frontier.put((priority, (new_state, exp_cost_h)))

    priority__state_h = frontier.get()

    current_state_exp_cost_h=priority__state_h[1]

    current_state=current_state_exp_cost_h[0]
    exp_cost_h=current_state_exp_cost_h[1]

    #priority=priority_and_state[0] #not necessary



print(current_state)
print(len(current_state.taken))
print(f"Solved in {counter:,} steps")
print(goal_check(current_state))


State(taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 18, 24, 51, 64, 73, 80, 81, 82, 83, 85, 89, 92, 93, 95}, not_taken={12, 13, 17, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 84, 86, 87, 88, 90, 91, 94, 96, 97, 98, 99})
29
Solved in 199 steps
True


In [6]:
#fail not usually reliable
def g(state):
    length = len(state.taken)
    return length**1.1 # find out that this can bring better results

def h(state):
    set_covered=reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))
    return PROBLEM_SIZE-np.count_nonzero(set_covered)

frontier = PriorityQueue()
initial_state = State(set(), set(range(NUM_SETS)))
frontier.put((0,(initial_state,PROBLEM_SIZE)))

counter=0
current_state = frontier.get()[1][0]
exp_cost_h = PROBLEM_SIZE

while exp_cost_h > 0:
    counter += 1
    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})

        act_cost_g = g(current_state)
        exp_cost_h = h(current_state)
        priority = act_cost_g+exp_cost_h
        frontier.put((priority, (new_state, exp_cost_h)))

    priority__state_h = frontier.get()

    current_state_exp_cost_h=priority__state_h[1]

    current_state=current_state_exp_cost_h[0]
    exp_cost_h=current_state_exp_cost_h[1]

    #priority=priority_and_state[0] #not necessary



print(current_state)
print(len(current_state.taken))
print(f"Solved in {counter:,} steps")
print(goal_check(current_state))


KeyboardInterrupt: 

In [12]:
#norm_factor = np.mean(np.array([np.count_nonzero(SETS[i]) for i in range(NUM_SETS)]))

#can introduce an exponential in cost that grows really faster aftwer the len of solution of greedy algorithm

def g(state):
    n=len(state.taken)
    m=NUM_SETS-n
    d=NUM_SETS/greedy_best
    Padd=(1-(1-P_TILE)**(m*(PROBLEM_SIZE-d*n)))
    print(Padd)
    return n*Padd

def h(state):
    set_covered=reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))
    return PROBLEM_SIZE-np.count_nonzero(set_covered)

frontier = PriorityQueue()
initial_state = State(set(), set(range(NUM_SETS)))
frontier.put((0,(initial_state,PROBLEM_SIZE)))

counter=0
current_state = frontier.get()[1][0]
exp_cost_h = PROBLEM_SIZE

while exp_cost_h > 0:
    counter += 1
    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})

        act_cost_g = g(current_state)
        exp_cost_h = h(current_state)
        priority = act_cost_g+exp_cost_h
        frontier.put((priority, (new_state, exp_cost_h)))

    priority__state_h = frontier.get()

    current_state_exp_cost_h=priority__state_h[1]

    current_state=current_state_exp_cost_h[0]
    exp_cost_h=current_state_exp_cost_h[1]

    #priority=priority_and_state[0] #not necessary



print(current_state)
print(len(current_state.taken))
print(f"Solved in {counter:,} steps")
print(goal_check(current_state))


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
